In [11]:
import albumentations as A
import cv2 as cv
import os

In [12]:
'''
Reads in a text file as a list of lists for data augmentation
Expects a filepath to the text file and the delimiter
'''
def readList(filepath, delimiter=','):
    output = []
    with open(filepath, 'r') as f:
        for line in f:
            output.append(line.strip().split(delimiter))
        return output
    
'''
Takes list of lists and separates class label from bounding box
Expects a list of bounding boxes:
where index 0 is the class label,
and the bounding boxes are eligible to be converted to a float
example of a labels:
[['0', '0.1859375', '0.49296875', '0.14453125', '0.959375'],
 ['0', '0.39375', '0.496875', '0.140625', '0.96015625'],
 ['0', '0.60546875', '0.496875', '0.1421875', '0.96796875'],
 ['0', '0.81328125', '0.4984375', '0.1421875', '0.9671875']]
'''
def processLabels(labels):
    classLabels = []
    bboxes = []
    for box in labels:
        classLabels.append(box[0])
        box.pop(0)
        bboxes.append([float(i) for i in box])
    return classLabels, bboxes

'''
Takes an image bounding boxes and classLabels and draws them on the image
'''
def drawBoundingBox(img, bboxes, classLabels):
    h, w, _ = img.shape
    
    for i, box in enumerate(bboxes):
        x_center, y_center, box_width, box_height = box
        x_min = int((x_center - box_width / 2) * w)
        y_min = int((y_center - box_height / 2) * h)
        x_max = int((x_center + box_width / 2) * w)
        y_max = int((y_center + box_height / 2) * h)
    
        # Draw the bounding box
        cv.rectangle(img, (x_min, y_min), (x_max, y_max), (0, 0, 255), 2)
    
        # Calculate the position for the class label
        label_x = x_min
        label_y = y_min - 10  # Move slightly higher above the bounding box
    
        # If the label is too close to the top, place it below the box instead
        if label_y < 10:
            label_y = y_min + 20
    
        # Add the class label with reduced thickness
        cv.putText(img, classLabels[i], (label_x, label_y), cv.FONT_HERSHEY_DUPLEX, 0.5, (0, 0, 255), 1)  # Reduce thickness to 1
    
    return img

'''
Combines each element from A into the first index of each list in B
A should be a list
B should be a list of lists
A and B should be the same length
'''
def combineLists(A, B):
    result = []
    for a, b in zip(A, B):
        combined = [a] + b
        result.append(combined)
    return result

## Transform Single Image

In [13]:
# # Read in image
# img = cv.imread('Similar Fork.v2i.yolov11/train/images/ABCT3883_JPEG.rf.0107c459d147343004a61a3820436508.jpg')
# 
# # Read in bounding box
# labels = readList('Similar Fork.v2i.yolov11/train/labels/ABCT3883_JPEG.rf.0107c459d147343004a61a3820436508.txt', ' ')
# 
# # Separate class label from bounding boxes
# classLabels, bboxes = processLabels(labels)
# 
# print(classLabels)
# print(bboxes)
# 
# # Draw original bounding box
# origImg = drawBoundingBox(img.copy(), bboxes, classLabels)
# cv.imshow('original img', origImg)
# cv.waitKey(1)
# 
# # Pipline for augmentation
# transform = A.Compose([
#     A.HorizontalFlip(p=1),
# ], bbox_params=A.BboxParams(format='yolo', label_fields=['classLabels']))
# 
# # Perform transformation
# transformed = transform(image=img, bboxes=bboxes, classLabels=classLabels)
# transformedImage = transformed['image']
# transformedBBoxes = transformed['bboxes']
# transformedClassLabels = transformed['classLabels']
# 
# # Show the transformed image and bounding boxes
# newImage = drawBoundingBox(transformedImage, transformedBBoxes, transformedClassLabels)
# cv.imshow('transformed img', newImage)
# cv.waitKey(0)
# cv.destroyAllWindows()

## Mass Data Augmentation

In [14]:
# Individual transforms to apply
transforms = [
    A.HorizontalFlip(p=1),
    A.RandomBrightnessContrast(p=1),
    A.RandomBrightnessContrast(p=1),
    A.RandomBrightnessContrast(p=1),
    A.VerticalFlip(p=1),
    A.Blur(blur_limit=7, p=1),
    A.GaussNoise(p=1),
    A.CLAHE(p=1),
    A.ChannelDropout(p=1),
    A.ColorJitter(p=1),
    A.Defocus(p=1),
    A.Downscale(p=1),
    A.Emboss(p=1),
    A.GaussNoise(p=1, noise_scale_factor=.1),
    # A.GlassBlur(p=1),
    # A.InvertImg(p=1),
    A.MedianBlur(p=1),
    A.MotionBlur(p=1),
    A.Posterize(p=1),
    A.RGBShift(p=1),
    # A.RandomGravel(p=1),
    # A.RandomRain(p=1),
    # A.RandomSnow(p=1),
    # A.RandomSunFlare(p=1, src_radius=150),
    A.Sharpen(p=1),
    # A.Solarize(p=1),
    A.Spatter(p=1),
    A.Superpixels(p=1),
    # A.ToGray(p=1),
    # A.ToSepia(p=1),
    # A.CoarseDropout(p=1, num_holes_range=(1, 500)),
    A.Perspective(p=1),
    # A.PixelDropout(p=1),
]

In [15]:
# MAKE SURE THERE IS A '/' AT THE END OF THE FILE PATH
TARGETIMGS = 'C:/Users/Trever/Desktop/Dataset/Versions/500x22/Similar Fork.v4i.yolov11/valid/images/'  # Location of images that need to be augmented
TARGETLBS = 'C:/Users/Trever/Desktop/Dataset/Versions/500x22/Similar Fork.v4i.yolov11/valid/labels/' # Location of labels that need to be augmented
DESTIMGS = 'C:/Users/Trever/Desktop/Dataset/Versions/500x22/Similar Fork.v4i.yolov11/valid/new_images/'  # Location of augmented images to be saved
DESTILBS = 'C:/Users/Trever/Desktop/Dataset/Versions/500x22/Similar Fork.v4i.yolov11/valid/new_labels/'  # Location of augmented labels to be saved

# Make lists with filenames
imgFileNames = [f for f in sorted(os.listdir(TARGETIMGS))]
labelFileNames = [f for f in sorted(os.listdir(TARGETLBS))]

for i in range(len(imgFileNames)):
    # Get each img and label
    imgPath = TARGETIMGS + imgFileNames[i]
    img = cv.imread(imgPath)
    labels = readList(TARGETLBS + labelFileNames[i], ' ')
    
    # Separate class label from bounding boxes
    classLabels, bboxes = processLabels(labels)
    
    # Draw original bounding box
    # origImg = drawBoundingBox(img.copy(), bboxes, classLabels)
    # cv.imshow('original img', origImg)
    # cv.waitKey(0)
    
    try:
        # Apply each transformation individually and save the results
        for j, transform in enumerate(transforms):
            compose = A.Compose([
                transform,
                ], bbox_params=A.BboxParams(format='yolo', label_fields=['classLabels']))
            transformed = compose(image=img.copy(), bboxes=bboxes, classLabels=classLabels)
            transformedImage = transformed['image']
            transformedBBoxes = transformed['bboxes']
            transformedClassLabels = transformed['classLabels']
            
            # Put class labels back in bounding boxes
            transformedLabels = combineLists(transformedClassLabels, transformedBBoxes)
        
            # newImage = drawBoundingBox(transformedImage, transformedBBoxes, transformedClassLabels)
            # cv.imshow(f'transform {j + 1}: {transform}', newImage)
            # cv.waitKey(0)
        
            # Save the augmented image and labels
            outputFileNameImg = f'{DESTIMGS}{imgFileNames[i].replace('.jpg', '')}{j}.jpg'
            outputFileNameLabels = f'{DESTILBS}{labelFileNames[i].replace('.txt', '')}{j}.txt'
            cv.imwrite(outputFileNameImg, transformedImage)
    
            # Write labels to file
            with open(outputFileNameLabels, 'w') as f:
                for row in transformedLabels:
                    f.write(' '.join(map(str, row)) + '\n')
    except:
        print(f'{imgFileNames[i]} could not be processed')
    # cv.destroyAllWindows()